# Data Cleaning

In [2]:
import pandas as pd
import numpy as np

In [3]:
def read_data(xls_path):
    xls = pd.ExcelFile(xls_path)
    if (xls_path == "../Ga Tech/GA Tech Data Final.xlsx"):
        recipe_ingredients = pd.read_excel(xls, 'Ingredients', index_col=None, header=None)
        recipe_type = pd.read_excel(xls,"Recipe Type", index_col=None, header=None)
        occasion = pd.read_excel(xls,"Occasion", index_col=None, header=None)
        course = pd.read_excel(xls,"Course", index_col=None, header=None)
        diet = pd.read_excel(xls, "Diet", index_col=None, header=None, parse_cols=[0,1,3])
        ethnicity = pd.read_excel(xls, "Ethnicity", index_col=None, header=None)
    else:
        recipe_ingredients = pd.read_excel(xls, 'Ingredients', index_col=None, header=None, parse_cols=[0,1,2])
        recipe_type = pd.read_excel(xls,"Recipe Type", index_col=None, header=None, parse_cols=[0,1,2])
        occasion = pd.read_excel(xls,"Occasion", index_col=None, header=None)
        course = pd.read_excel(xls,"Course", index_col=None, header=None, parse_cols=[0,1,2])
        diet = pd.read_excel(xls, "Diet", index_col=None, header=None, parse_cols=[0,1,2])
        ethnicity = pd.read_excel(xls, "Ethnicity", index_col=None, header=None, parse_cols=[0,1,2])
    recipe_ingredients.columns = ["id","recipe_name","ingredients"]
    recipe_type.columns = ["id","recipe_name","recipe_type"]
    occasion.columns = ["id","recipe_name","occasion"]
    course.columns = ["id","recipe_name","course"]
    diet.columns = ["id","recipe_name","diet"]
    ethnicity.columns = ["id","recipe_name","ethnicity"]
    return recipe_ingredients, recipe_type, occasion, course, diet, ethnicity

def row_concat(df1, df2):
    return pd.concat([df1,df2], axis=0)

In [4]:
recipe_ingredients_1, recipe_type_1, occasion_1, course_1, diet_1, ethnicity_1 = read_data("GA Tech Data Final.xlsx")
recipe_ingredients_2, recipe_type_2, occasion_2, course_2, diet_2, ethnicity_2 = read_data("GA Tech Data Final 2.xlsx")

In [5]:
recipe_ingredients = row_concat(recipe_ingredients_1, recipe_ingredients_2)
recipe_type = row_concat(recipe_type_1, recipe_type_2)
occasion = row_concat(occasion_1, occasion_2)
course = row_concat(course_1, course_2)
diet = row_concat(diet_1, diet_2)
ethnicity = row_concat(ethnicity_1, ethnicity_2)

In [6]:
recipe_ingredients.loc[recipe_ingredients.ingredients.isnull(),"ingredients"] = ["cheddar cheese","smoked cheddar cheese","croissants","cream","Vegemite"]
recipe_type.loc[recipe_type.recipe_type.isnull(),"recipe_type"] = "Sandwiches & burgers"
course.loc[course.course.isnull(),"recipe_type"] = "Breakfast / brunch"
diet.loc[diet.diet.isnull(), "diet"] = "Vegetarian"
ethnicity.loc[ethnicity.ethnicity.isnull(), "ethnicity"] = "French"
recipe_index = recipe_ingredients.loc[:,["id","recipe_name"]]
recipe_index.drop_duplicates(inplace=True)
recipe_index.reset_index(inplace=True)
recipe_index.drop(['index'],axis=1,inplace=True)
print recipe_index.head()

       id                                        recipe_name
0  508083                                Tuscan potato chips
1  508095                           Stuffed zucchini flowers
2  508101                                  Fried little fish
3  508106                                     Peach fritters
4  508120  Goat cheese tart with caramelized onions and g...


In [ ]:
# diet.diet.unique()
# recipe_type.recipe_type.unique()
# occasion.occasion.unique()
# course.course.unique()
# ethnicity.ethnicity.unique()

In [7]:
#drop recipe name column, which is unnecessary in db
diet.drop("recipe_name", axis=1, inplace=True)
recipe_type.drop("recipe_name", axis=1, inplace=True)
occasion.drop("recipe_name", axis=1, inplace=True)
course.drop("recipe_name", axis=1, inplace=True)
ethnicity.drop("recipe_name", axis=1, inplace=True)
recipe_ingredients.drop("recipe_name", axis=1, inplace=True)

In [ ]:
# turn recipe-tag format into one-row-per-recipe-with-column-for-each-feature format
# get unique values for each feature and transform
diet_dummies = pd.get_dummies(diet,columns=["diet"]).groupby(["id"]).sum().reset_index()
recipe_type_dummies = pd.get_dummies(recipe_type, columns=["recipe_type"]).groupby(["id"]).sum().reset_index()
course_dummies = pd.get_dummies(course, columns=["course"]).groupby(["id"]).sum().reset_index()
ethnicity_dummies = pd.get_dummies(ethnicity, columns=['ethnicity']).groupby(["id"]).sum().reset_index()
occasion_dummies = pd.get_dummies(occasion, columns=['occasion']).groupby(["id"]).sum().reset_index()

In [8]:
#generate unique ingredients table/ids
unique_ingredients = {value:key for (key, value) in enumerate(recipe_ingredients.ingredients.unique())}
recipe_ingredients["ingredient_id"] = recipe_ingredients.ingredients\
    .apply(lambda x: unique_ingredients[x])
ingredients = pd.DataFrame([(value, key) for key, value in unique_ingredients.iteritems()],\
                          columns=["id", "ingredient"])
recipe_ingredients.drop("ingredients", axis=1, inplace=True)

# Database Connection

In [9]:
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine 
import sqlite3

In [10]:
engine = create_engine('sqlite:///chefs.db')

In [24]:
def conn_sqlite(df, name=None, con=None, if_exists='append'):

    if if_exists=='replace' and table_exists(name, con):    
        cur = con.cursor()   
        cur.execute("drop table "+name)
        cur.close()    
        
    if if_exists in ('fail','replace') or ( if_exists=='append' and table_exists(name, con)==False ):
        #create table
        schema = get_schema(df, name)
        cur = con.cursor()
        print schema
        cur.execute(schema)
        cur.close()
        #print 'created table' 

    cur = con.cursor()
    #bulk insert   
    wildcards = ','.join(['?'] * len(df.columns))
    insert_sql = 'INSERT OR IGNORE INTO %s VALUES (%s)' % (name, wildcards)
    print 'insert_sql', insert_sql
    data = [tuple(x) for x in df.values]
    print 'data', data[0:5]
    cur.executemany(insert_sql, data)  
      
    con.commit()
    cur.close()
    return

def table_exists(name=None, con=None):
    sql="SELECT name FROM sqlite_master WHERE type='table' AND name='MYTABLE';".replace('MYTABLE', name)    
    df = read_db(sql, con)
    #print sql, df
    print 'table_exists?', len(df)
    exists = True if len(df)>0 else False
    return exists

def read_db(sql, con):
    return pd.io.sql.read_sql_query(sql, con)



def get_schema(df, name):
    types = {'INT':'NUMBER',  'FLOAT':'NUMBER', 'VARCHAR':'VARCHAR2'}
    column_types = []
    dtypes = df.dtypes
    for i,k in enumerate(dtypes.index):
        dt = dtypes[k]
        if issubclass(dt.type, (np.integer, np.bool_)):
            sqltype = types['INT']
        elif issubclass(dt.type, np.floating):
            sqltype = types['FLOAT']
        else:
            sampl = df[ df.columns[i] ][0]
            sqltype = types['VARCHAR']
        colname =  k.replace('-','_').strip()  
        colname = colname.replace(' ','_').strip()
        column_types.append((colname, sqltype))
    columns = ',\n  '.join('%s %s' % x for x in column_types)
    template_create = """CREATE TABLE %(name)s (%(columns)s"""    
    #print 'COLUMNS:\n', columns
    create = template_create % {'name' : name, 'columns' : columns}
    
    #hardcode primary/foreign key references per table
    if name == "recipe_index":
        create += ',\n  PRIMARY KEY(id)'
    elif name == "diet":
        create += ',\n  FOREIGN KEY (id) REFERENCES recipe_index (id)'
    elif name == "recipeType":
        create += ',\n  FOREIGN KEY (id) REFERENCES recipe_index (id)'
    elif name == "course":
        create += ',\n  FOREIGN KEY (id) REFERENCES recipe_index (id)'
    elif name == "ethnicity":
        create += ',\n  FOREIGN KEY (id) REFERENCES recipe_index (id)'
    elif name == "occasion":
        create += ',\n  FOREIGN KEY (id) REFERENCES recipe_index (id)'
    elif name == "recipe_ingredients":
        create += ',\n  FOREIGN KEY (id) REFERENCES recipe_index (id)'
        create += ',\n  FOREIGN KEY (ingredient_id) REFERENCES ingredients (id)'
    elif name == "ingredients":
        create += ',\n  PRIMARY KEY(id)'
    create += ');'
    return create

In [12]:
# build sql engine and create connection
con = engine.raw_connection()

In [25]:
#recipe table
conn_sqlite(recipe_index,'recipe_index', con, "replace")

table_exists? 1
CREATE TABLE recipe_index (id NUMBER,
  recipe_name VARCHAR2,
  PRIMARY KEY(id));
insert_sql INSERT OR IGNORE INTO recipe_index VALUES (?,?)
data [(508083, u'Tuscan potato chips'), (508095, u'Stuffed zucchini flowers'), (508101, u'Fried little fish'), (508106, u'Peach fritters'), (508120, u'Goat cheese tart with caramelized onions and green olives')]


In [26]:
#diet table
conn_sqlite(diet,'diet',con,"replace")

table_exists? 1
CREATE TABLE diet (id NUMBER,
  diet VARCHAR2,
  FOREIGN KEY (id) REFERENCES recipe_index (id));
insert_sql INSERT OR IGNORE INTO diet VALUES (?,?)
data [(508083, 'Vegetarian'), (508095, 'Vegetarian'), (508120, 'Vegetarian'), (508132, 'Vegetarian'), (508132, 'Vegetarian')]


In [27]:
#recipe type table
conn_sqlite(recipe_type,'recipeType',con,"replace")

table_exists? 1
CREATE TABLE recipeType (id VARCHAR2,
  recipe_type VARCHAR2,
  FOREIGN KEY (id) REFERENCES recipe_index (id));
insert_sql INSERT OR IGNORE INTO recipeType VALUES (?,?)
data [(u'No', u'Recipe Type'), (1315553, u'Sandwiches & burgers'), (1304053, u'Sauces, general'), (1304889, u'Dips, spreads & salsas'), (1304874, u'Baked & steamed desserts')]


In [28]:
#course table
conn_sqlite(course,'course',con,"replace")

table_exists? 1
CREATE TABLE course (id NUMBER,
  course VARCHAR2,
  recipe_type VARCHAR2,
  FOREIGN KEY (id) REFERENCES recipe_index (id));
insert_sql INSERT OR IGNORE INTO course VALUES (?,?,?)
data [(1274385, u'Appetizers / starters', nan), (1259764, u'Main course', nan), (1372473, u'Appetizers / starters', nan), (1372473, u'Small plates - tapas, meze', nan), (1319144, u'Appetizers / starters', nan)]


In [29]:
#ethnicity table
conn_sqlite(ethnicity,'ethnicity',con,"replace")

table_exists? 1
CREATE TABLE ethnicity (id NUMBER,
  ethnicity VARCHAR2,
  FOREIGN KEY (id) REFERENCES recipe_index (id));
insert_sql INSERT OR IGNORE INTO ethnicity VALUES (?,?)
data [(508083, u'Italian'), (508095, u'Italian'), (508133, u'French'), (508143, u'German'), (508143, u'Austrian')]


In [30]:
#occasion table
conn_sqlite(occasion,'occasion',con,"replace")

table_exists? 1
CREATE TABLE occasion (id NUMBER,
  occasion VARCHAR2,
  FOREIGN KEY (id) REFERENCES recipe_index (id));
insert_sql INSERT OR IGNORE INTO occasion VALUES (?,?)
data [(508333, u'Fall / autumn'), (508336, u'Summer'), (508611, u'Spring'), (508623, u'Cooking ahead'), (661952, u'Dinner parties/entertaining')]


In [31]:
#recipe_ingredients junction table
conn_sqlite(recipe_ingredients,'recipe_ingredients',con,"replace")

table_exists? 1
CREATE TABLE recipe_ingredients (id NUMBER,
  ingredient_id NUMBER,
  FOREIGN KEY (id) REFERENCES recipe_index (id),
  FOREIGN KEY (ingredient_id) REFERENCES ingredients (id));
insert_sql INSERT OR IGNORE INTO recipe_ingredients VALUES (?,?)
data [(508083, 0), (508083, 1), (508083, 2), (508095, 3), (508095, 4)]


In [32]:
#ingredients table
conn_sqlite(ingredients,'ingredients',con,"replace")

table_exists? 1
CREATE TABLE ingredients (id NUMBER,
  ingredient VARCHAR2,
  PRIMARY KEY(id));
insert_sql INSERT OR IGNORE INTO ingredients VALUES (?,?)
data [(8571, u'coarse cereal'), (8234, u'butter flavoring'), (8084, u'Butterfinger candy bars'), (4724, u'bucatini pasta'), (3763, u'provolone piccante cheese')]


In [36]:
ingredients.head(40)

,id,ingredient
0,8571,coarse cereal
1,8234,butter flavoring
2,8084,Butterfinger candy bars
3,4724,bucatini pasta
4,3763,provolone piccante cheese
5,12669,wild hibiscus flowers in syrup
6,10202,Bacardi Superior rum
7,9453,beef middle casings
8,7805,fermented beet juice
9,5661,stick canela
